In [1]:
import itertools, matplotlib, pandas, numpy as np
import matplotlib.pyplot as plt
from bokeh.layouts import column, row, widgetbox
from bokeh.models import CustomJS, ColumnDataSource, Slider, Range1d, LabelSet, Label, Arrow, OpenHead, NormalHead, VeeHead
from bokeh.plotting import figure, output_file, show, curdoc, output_notebook
from bokeh.themes import Theme
import yaml
output_notebook()

Loading BokehJS ...

In [3]:
met_allowed = [-1.1, -1.0, -0.9, -0.8, -0.7, -0.6, -0.5,
               -0.4, -0.3, -0.2, -0.1, 0., 0.1, 0.2]
age_allowed = [1., 2., 3., 4., 5., 6., 7., 8., 9., 
               10., 11., 12., 13.]

In [4]:
allspec = []

for age, met in list(itertools.product(age_allowed, met_allowed)):
    # File names
    fname1 = 'model/cont_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'
    fname2= 'model/normcont4047_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'
    fname3 = 'model/normcont4757_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'
    fname4 = 'model/normcont5767_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'
    fname5 = 'model/normcont8089_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'
    fname6 = 'model/normcont9010_z' + str(round(met, 1)) + '_age' + \
        str(round(age, 1)) + '.dat'

    wave = pandas.read_csv(fname1)['wave'].values
    flux1 = pandas.read_csv(fname1)['flux'].values
    flux2 = pandas.read_csv(fname2)['flux'].values
    flux3 = pandas.read_csv(fname3)['flux'].values
    flux4 = pandas.read_csv(fname4)['flux'].values
    flux5 = pandas.read_csv(fname5)['flux'].values
    flux6 = pandas.read_csv(fname6)['flux'].values
    
    temdata = np.vstack((wave, flux1, flux2, 
                         flux3, flux4, flux5, flux6))
    
    #allspec.append(
    #    {'param': 10000*int(age)+int(met*10), 'wave': wave,
    #     'flux1': flux1, 'flux2': flux2, 'flux3': flux3,
    #     'flux4': flux4, 'flux5': flux5, 'flux6': flux6})
    allspec.append(
        {'param': 10000*int(age)+int(met*10), 'wave': wave,
         'data': temdata})

In [6]:
specdata = pandas.DataFrame(allspec)
spec_cds = ColumnDataSource(specdata)

In [18]:
def test(doc):
    # default spectrum
    data0 = spec_cds.data['data'][spec_cds.data['param']==10000*int(1.0)+int(-1.0*10)][0]
    spec_plot = ColumnDataSource(
        data=dict(wave=data0[0], flux1=data0[1]))
    
    s0 = figure(x_range=(3500, 1.1e4), plot_width=800, plot_height=200, title=None)
    s0.line('wave', 'flux1', source=spec_plot)

    def callback(attr, old, new, spec_cds=spec_cds):
        tage = age.value
        zh = zmet.value
        datax = spec_cds.data['data'][spec_cds.data['param']==10000*int(tage)+int(zh*10)][0]
        spec_plot.data = dict(wave=datax[0], flux1=datax[1])
        
        
    age_slider = Slider(title="age", value=1., start=1., end=13., step=1.)
    zmet_slider = Slider(title="[Z/H]", value=-1., start=-1.1, end=0.2, step=0.1)
    callback.args["age"] = age_slider
    callback.args["met"] = zmet_slider
    
    for w in [age, zmet]:
        w.on_change('value', callback)

    layout = column(
        widgetbox(age, zmet),s0)

    doc.add_root(layout)

In [19]:
show(test) # notebook_url="http://localhost:8888"

In [51]:
for w in [age, zmet]:
        w.on_change('value', callback)

layout = column(
        widgetbox(age, zmet),s0)

curdoc().add_root(layout)

In [ ]:
def alf_interact(doc):
    f0 = Spectra[str(10000*int(-1*10)+int(1))]  
    source = ColumnDataSource(data=dict(x=f0[0], y0=f0[1], y1=f0[2], y2=f0[3], y3=f0[4], y4=f0[5], y5=f0[6]))

    s0 = figure(x_range=(3500, 1.1e4), plot_width=800, plot_height=200, title=None)
    s1 = figure(x_range=(4e3, 4.7e3), y_range=(0.6, 1.25), plot_width=250, plot_height=200, title=None)
    s2 = figure(x_range=(4.7e3, 5.7e3), y_range=(0.6, 1.2), plot_width=250, plot_height=200, title=None)
    s3 = figure(x_range=(5.7e3, 6.7e3), y_range=(0.6, 1.2), plot_width=250, plot_height=200, title=None)
    s4 = figure(x_range=(8e3, 8.92e3), y_range=(0.75, 1.1), plot_width=390, plot_height=200, title=None)
    s5 = figure(x_range=(9e3, 1.01e4), y_range=(0.9, 1.1), plot_width=390, plot_height=200, title=None)
    for i, tem in enumerate([s0, s1, s2, s3, s4, s5]):
        tem.xaxis.axis_label = "Wavelength"
        tem.yaxis.axis_label = "Flux"
    for i, tem in enumerate([s1, s2, s3, s4, s5]):
        tem.yaxis.axis_label = "Continuum Normed\nFlux"
    for i, tem in enumerate([s0, s1, s2, s3, s4, s5]):
        tem.line('x', 'y'+str(i), source=source, line_width=1, line_alpha=0.75, line_color='black')

    age = Slider(title="age", value=1., start=1., end=13., step=1.)
    zmet = Slider(title="[Z/H]", value=-1., start=-1.1, end=0.2, step=0.1)

    def callback(attrname, old, new):
        tage = age.value
        zh = zmet.value
        fx = Spectra[str(10000*int(zh*10)+int(tage))]
        source.data = dict(x=fx[0], y0=fx[1], y1=fx[2], y2=fx[3], y3=fx[4], y4=fx[5], y5=fx[6])
        
    for w in [age, zmet]:
        w.on_change('value', callback)

    layout = column(
        widgetbox(age, zmet),
        s0, row(s1, s2, s3), row(s4, s5))

    doc.add_root(layout)
    
    #myplot_html = file_html(layout, CDN)
    # this HTML code is very long (~30 K), the cell below doesn't show all the code in NBviewer
    #print(myplot_html)
    #HTML(myplot_html)

    #output_file("test.html")
    #save(doc)
    
    textlist = ['G4300', "Hgamma"]
    for i, xloc in enumerate([(4281.375+4316.375)/2, (4331.25+4352.25)/2,]):
        citation = Label(x=xloc, y=1.18+(-1)**i*0.02, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s1.add_layout(citation)
        s1.line([xloc, xloc], [1.14, 1.18])
                
    textlist = ['Hbeta', 'Mgb', 'Fe5270', 'Fe5335']
    for i, xloc in enumerate([(4847.875+4876.625)/2, (5160.125+5192.625)/2, (5245.650+5285.65)/2, (5312.125+5352.125)/2]):
        citation = Label(x=xloc, y=1.14+(-1)**i*0.02, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s2.add_layout(citation)
        s2.line([xloc, xloc], [1.10, 1.14])
                
    textlist = ['NaD', 'TiO', 'TiO2',]
    for i, xloc in enumerate([(5876.875+5909.375)/2.,(5936.625+5994.125)/2., (6189.625+6272.125)/2.,]):
        citation = Label(x=xloc, y=1.16-(-1)**i*0.015, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s3.add_layout(citation)
        s3.line([xloc, xloc], [1.10, 1.14])
        
    textlist = ['CaT',]
    for i, xloc in enumerate([8498, 8542, 8662]):
        if i==1:
            citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                     text=textlist[0], render_mode='css', text_font_size='7pt')
            s4.add_layout(citation)
        s4.line([xloc, xloc], [1.05, 1.08])
        
    textlist = ['NaI',]
    for i, xloc in enumerate([(8168.5+8234.125)/2.,]):
        citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s4.add_layout(citation)
        s4.line([xloc, xloc], [1.06, 1.08])
        
    textlist = ['FeH',]
    for i, xloc in enumerate([9920,]):
        citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s5.add_layout(citation)
        s5.line([xloc, xloc], [1.06, 1.08])

In [2]:
Spectra = {}
    
for zmet in [-1.1, -1.0, -0.9, -0.8, -0.7, -0.6, -0.5, -0.4, -0.3, -0.2, -0.1, 0., 0.1, 0.2]:
    for age in [1., 2., 3., 4., 5., 6., 7., 8., 9., 10., 11., 12., 13.]:
        fname = 'model/cont_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        fname1 = 'model/normcont4047_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        fname2 = 'model/normcont4757_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        fname3 = 'model/normcont5767_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        fname4 = 'model/normcont8089_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        fname5 = 'model/normcont9010_z'+str(round(zmet,1))+'_age'+str(round(age,1))+'.dat'
        Spectra[str(10000*int(zmet*10)+int(age))]=np.vstack((pandas.read_csv(fname)['wave'], pandas.read_csv(fname)['flux'], 
                                                 pandas.read_csv(fname1)['flux'], pandas.read_csv(fname2)['flux'], 
                                                 pandas.read_csv(fname3)['flux'], pandas.read_csv(fname4)['flux'], 
                                                 pandas.read_csv(fname5)['flux']))

In [143]:
def alf_interact(doc):
    f0 = Spectra[str(10000*int(-1*10)+int(1))]  
    source = ColumnDataSource(data=dict(x=f0[0], y0=f0[1], y1=f0[2], y2=f0[3], y3=f0[4], y4=f0[5], y5=f0[6]))

    s0 = figure(x_range=(3500, 1.1e4), plot_width=800, plot_height=200, title=None)
    s1 = figure(x_range=(4e3, 4.7e3), y_range=(0.6, 1.25), plot_width=250, plot_height=200, title=None)
    s2 = figure(x_range=(4.7e3, 5.7e3), y_range=(0.6, 1.2), plot_width=250, plot_height=200, title=None)
    s3 = figure(x_range=(5.7e3, 6.7e3), y_range=(0.6, 1.2), plot_width=250, plot_height=200, title=None)
    s4 = figure(x_range=(8e3, 8.92e3), y_range=(0.75, 1.1), plot_width=390, plot_height=200, title=None)
    s5 = figure(x_range=(9e3, 1.01e4), y_range=(0.9, 1.1), plot_width=390, plot_height=200, title=None)
    for i, tem in enumerate([s0, s1, s2, s3, s4, s5]):
        tem.xaxis.axis_label = "Wavelength"
        tem.yaxis.axis_label = "Flux"
    for i, tem in enumerate([s1, s2, s3, s4, s5]):
        tem.yaxis.axis_label = "Continuum Normed\nFlux"
    for i, tem in enumerate([s0, s1, s2, s3, s4, s5]):
        tem.line('x', 'y'+str(i), source=source, line_width=1, line_alpha=0.75, line_color='black')

    age = Slider(title="age", value=1., start=1., end=13., step=1.)
    zmet = Slider(title="[Z/H]", value=-1., start=-1.1, end=0.2, step=0.1)

    def callback(attrname, old, new):
        tage = age.value
        zh = zmet.value
        fx = Spectra[str(10000*int(zh*10)+int(tage))]
        source.data = dict(x=fx[0], y0=fx[1], y1=fx[2], y2=fx[3], y3=fx[4], y4=fx[5], y5=fx[6])
        
    for w in [age, zmet]:
        w.on_change('value', callback)

    layout = column(
        widgetbox(age, zmet),
        s0, row(s1, s2, s3), row(s4, s5))

    doc.add_root(layout)
    
    #myplot_html = file_html(layout, CDN)
    # this HTML code is very long (~30 K), the cell below doesn't show all the code in NBviewer
    #print(myplot_html)
    #HTML(myplot_html)

    #output_file("test.html")
    #save(doc)
    
    textlist = ['G4300', "Hgamma"]
    for i, xloc in enumerate([(4281.375+4316.375)/2, (4331.25+4352.25)/2,]):
        citation = Label(x=xloc, y=1.18+(-1)**i*0.02, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s1.add_layout(citation)
        s1.line([xloc, xloc], [1.14, 1.18])
                
    textlist = ['Hbeta', 'Mgb', 'Fe5270', 'Fe5335']
    for i, xloc in enumerate([(4847.875+4876.625)/2, (5160.125+5192.625)/2, (5245.650+5285.65)/2, (5312.125+5352.125)/2]):
        citation = Label(x=xloc, y=1.14+(-1)**i*0.02, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s2.add_layout(citation)
        s2.line([xloc, xloc], [1.10, 1.14])
                
    textlist = ['NaD', 'TiO', 'TiO2',]
    for i, xloc in enumerate([(5876.875+5909.375)/2.,(5936.625+5994.125)/2., (6189.625+6272.125)/2.,]):
        citation = Label(x=xloc, y=1.16-(-1)**i*0.015, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s3.add_layout(citation)
        s3.line([xloc, xloc], [1.10, 1.14])
        
    textlist = ['CaT',]
    for i, xloc in enumerate([8498, 8542, 8662]):
        if i==1:
            citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                     text=textlist[0], render_mode='css', text_font_size='7pt')
            s4.add_layout(citation)
        s4.line([xloc, xloc], [1.05, 1.08])
        
    textlist = ['NaI',]
    for i, xloc in enumerate([(8168.5+8234.125)/2.,]):
        citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s4.add_layout(citation)
        s4.line([xloc, xloc], [1.06, 1.08])
        
    textlist = ['FeH',]
    for i, xloc in enumerate([9920,]):
        citation = Label(x=xloc, y=1.08, x_units='data', y_units='data',
                 text=textlist[i], render_mode='css', text_font_size='7pt')
        s5.add_layout(citation)
        s5.line([xloc, xloc], [1.06, 1.08])

In [144]:
show(alf_interact) # notebook_url="http://localhost:8888"

In [8]:
import bokeh
from IPython.core.display import HTML
print('Bokeh version:', bokeh.__version__)
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.resources import CDN
from bokeh.embed import file_html
from IPython.core.display import HTML

('Bokeh version:', u'0.13.0')
